# Description
This will focus on using classic regression models and bayesian models. 

#### NOTE: As described in EDA notebook, "Pseudo_ts" is concatenation of data from locally adjacent ski resorts (e.g., all resorts in Colorado) into a single timeseries.
# Imports

In [69]:
! pip install vapeplot arviz pystan stan_utility

In [70]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    import os
    from google.colab import drive
    drive.mount('/content/gdrive')
    base_path = r'/content/gdrive/My Drive/data_sci/colab/ski/'
    os.chdir(base_path)
    try:
        ! git clone https://github.com/chrisoyer/ski-snow-modeling/
    except:  # if dir not empty e.g. already cloned
        ! git pullb
    mod_path = os.path.join(base_path, 
                            r"ski-snow-modeling/src/analysis/project_utils/project_utils.py")
    import importlib.util
    spec = importlib.util.spec_from_file_location(name="utils.name", location=mod_path)
    utils = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(utils)
    
    os.chdir('./ski-snow-modeling/src/analysis/')
    # Change the working directory to the repo root.
    # Add the repo root to the Python path.
    import sys
    sys.path.append(os.getcwd())
else:
    # local running
    import project_utils as utils

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'ski-snow-modeling' already exists and is not an empty directory.


In [71]:
# data wrangling
import numpy as np
import pandas as pd
import os.path
import pickle
import calendar

# viz
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import vapeplot
import arviz as az

# modeling
import pystan
import stan_utility
from sklearn.metrics import r2_score

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameters

In [72]:
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn')
plt.rc('figure', figsize=(11.0, 7.0))

# Load Data

In [73]:
file_path = r'../../data/snow_data_clean.parquet'
all_data_path = os.path.join(os.getcwd(), file_path)
model_path = r'./stan_model.pkl'
result_path = r'../../data/processed/stan_results.pkl'

In [13]:
# parquet opening is broken on colab
with open(file_path, 'rb') as parq_file:
    long_series_df = pd.read_parquet(parq_file)
assert long_series_df.base.isna().sum()==0

long_series_df.head()

,dayofyr,timestamp,base,station,snowfall,ski_yr,state,region,pseudo_ts_delt,pseudo_ski_yr,pseudo_ts,basecol_interpolated
11085,137.0,2016-01-10,0.000000,Mt. Holiday,2.0,5.0,michigan,Other,324.0,-31.0,1692-01-10,True
11086,138.0,2016-01-11,-2.320142,Mt. Holiday,3.0,5.0,michigan,Other,324.0,-31.0,1692-01-11,True
11087,139.0,2016-01-12,-2.320142,Mt. Holiday,5.0,5.0,michigan,Other,324.0,-31.0,1692-01-12,True
11088,140.0,2016-01-13,6.737995,Mt. Holiday,0.0,5.0,michigan,Other,324.0,-31.0,1692-01-13,False
11089,141.0,2016-01-14,10.000000,Mt. Holiday,0.0,5.0,michigan,Other,324.0,-31.0,1692-01-14,False


# Feature Engineering

In [14]:
def add_month(data: pd.DataFrame) -> pd.DataFrame:
    return data.assign(month=lambda x:
                       x.pseudo_ts.dt.month)

def add_diff(data: pd.DataFrame) -> pd.DataFrame:
    """ use difference in base, not absolute value """
    return (data
            .assign(delta_base=lambda x: x.base.diff(1))
            .fillna(0)
            .drop(columns=['base'])
           )

def ohe(data: pd.DataFrame, col: str) -> pd.DataFrame:
    return pd.concat([data.drop(columns=[col]),
                      pd.get_dummies(data[col],
                                     prefix=col)],
                     axis=1)

def add_month_x_snowfall(data: pd.DataFrame) -> pd.DataFrame:
    """adds interaction terms"""
    months = [col for col in data.columns
              if 'month_' in col]
    combos_df = pd.concat([pd.Series(data.snowfall * data[month],
                                     name='snowfall_x_' + month)
                           for month in months], axis=1)
    return pd.concat([data, combos_df], axis=1)

def cleaner(data: pd.DataFrame, includes: list=[None]) -> pd.DataFrame:
    """ Removes interpolated rows and unneeded columns
    Params:
        data: df to operate on
        includes: column names NOT to drop (don't need to specify usually)
    ski_yr is needed for test/train split"""
    data = data.query('basecol_interpolated==False')
    bad_cols = ['dayofyr', 'station', 'state', 'pseudo_ski_yr',
                'timestamp', 'basecol_interpolated', 'pseudo_ts',
                'pseudo_ts_delt'
               ]
    bad_cols = [col for col in bad_cols if col not in includes]
    return data.drop(columns=bad_cols)

In [15]:
data = (long_series_df.pipe(add_month)
        .pipe(add_diff)
        .pipe(ohe, 'month')
        .pipe(add_month_x_snowfall)
        .pipe(cleaner)
)
data.sum()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


snowfall                                                          239805
ski_yr                                                            847944
region                 OtherOtherOtherOtherOtherOtherOtherOtherOtherO...
delta_base                                                       57308.9
month_1                                                            53318
month_2                                                            47936
month_3                                                            45183
month_4                                                            14427
month_5                                                             1734
month_6                                                              357
month_7                                                               94
month_8                                                               14
month_9                                                                3
month_10                                           

In [77]:
long_series_df.station.value_counts().head(20)

Seven Springs          3693
Bristol Mountain       3670
Hilltop                3657
Mount Southington      3650
Yawgoo Valley          3648
Aspen / Snowmass       3647
Mt. Shasta Ski Park    3642
Chestnut Mountain      3636
White Pass             3636
Eldora                 3630
Diamond Peak           3623
Wachusett Mountain     3619
Sugar Bowl             3618
Hogadon Basin          3618
Monarch                3613
Kelly Canyon           3602
Silverton              3599
Lost Valley            3590
Heavenly               3390
Holiday Valley         3381
Name: station, dtype: int64

# Bayesian Model in Stan (MCMC)
I want to add priors to the model that snowfall should only result in increases in base depth, and monthly effects should only result in reduction (i.e., monthly effect should measure strength of melting.); changes at odds with this should be considered as noise. A bayesian model allows for this.

In [16]:
# set aside some data for multi-step analysis
leaveouts = ['Eldora', 'Seven Springs']
stan_multistep_test_df = long_series_df.query('station in @leaveouts')


stan_df = (long_series_df
           .pipe(add_month)
           .pipe(add_diff)
           .pipe(ohe, 'region')
           .pipe(ohe, 'month')
           .pipe(cleaner)
           )
stan_df.head()



,snowfall,ski_yr,delta_base,region_Cascades,region_Colorado,region_East,region_New_England,region_Other,region_Rockies_Other,region_Sierras,region_Utah,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
11088,0.0,5.0,9.058137,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
11089,0.0,5.0,3.262005,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
11090,0.0,5.0,0.000000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
11091,0.0,5.0,0.000000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
11092,0.0,5.0,0.000000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# sample data; half million records => slow mcmc

def sample_weighted_season(df: pd.DataFrame)->pd.DataFrame:
    """samples dataframe but doesn't remove rare months and mildly reduces
    amount of semi-rare months"""
    # un-OHE
    df['month'] = df[[c for c in data.columns if "month_" in c and "x_m" not in c]].idxmax(axis=1)
    # define months
    rare_months = [f'month_{i}' for i in range(5,11)]
    semirare_months = ['month_4', 'month_11']
    nonrare_months = ['month_12', 'month_1', 'month_2', 'month_3']
    # split and sample data
    rare_data = df.query('month in @rare_months')
    semirare_data = df.query('month in @semirare_months').sample(frac=.3, axis=0)
    nonrare_data = df.query('month in @nonrare_months').sample(frac=.09, axis=0)
    # recombine
    return pd.concat([rare_data, semirare_data, nonrare_data], axis=0).drop(columns=['month'])

# split data first so rare months get included in both sets
stan_sample_test_df = stan_df.sample(frac=.20, axis=0)
stan_sample_train_df = stan_df.drop(index=stan_sample_test_df.index)
# reduce data size 
stan_sample_test_df = stan_df.pipe(sample_weighted_season)
stan_sample_train_df = stan_df.pipe(sample_weighted_season)

In [18]:
# provide data including shapes and column type locations to stan
columns = stan_df.columns
region_cols = [c for c in columns if "region" in c]
month_cols = [col for col in stan_sample_train_df.columns if "month" in col]

def subsets(df: pd.DataFrame)-> tuple:
    X = df.drop(columns=['delta_base'])
    Xmonth= X[month_cols]
    Xsnow = X['snowfall']
    Xregion = X[region_cols]
    y = df[['delta_base']]
    return (X, Xmonth, Xsnow, Xregion, y)

X, X_month, X_snow, X_region, y = subsets(stan_sample_train_df)
X_test, X_month_test, X_snow_test, X_region_test, _ = subsets(stan_sample_test_df)

stan_data = {'N': X.shape[0],
             'K_month': X_month.shape[1],
             'X_month': X_month.to_numpy(),
             'K_reg': X_region.shape[1],
             'X_reg': X_region.to_numpy(),
             'X_snow': X_snow.to_numpy().reshape(-1,1),
             'y': y.to_numpy().reshape(-1),
             # test
             'N_test': X_test.shape[0],
             'X_month_test': X_month_test.to_numpy(),
             'X_reg_test': X_region_test.to_numpy(),
             'X_snow_test': X_snow_test.to_numpy().reshape(-1,1),
             }

In [21]:
stan_model_str = """
functions {}
data {
    // input data passed from Python
    int<lower=1> N;               // number of data observations
    int<lower=1> K_month;         // no of melting predictor
    matrix[N, K_month] X_month;   // predictor for melting features
    int<lower=1> K_reg;           // no of region features
    matrix[N, K_reg] X_reg;       // region predictors
    matrix[N, 1] X_snow;          // snowfall predictor
    vector[N] y;                  // response vector
    
    // test variables
    int<lower=1> N_test;                  // no of test records
    matrix[N_test, K_month] X_month_test; // predictor for melting features
    matrix[N_test, K_reg] X_reg_test;     // region predictors
    matrix[N_test, 1] X_snow_test;
}
transformed data {
    matrix[N, K_reg] X_reg_snow;
    row_vector[N] X_snow_rvect = to_row_vector(X_snow);
    matrix[N_test, K_reg] X_reg_snow_test;
    row_vector[N_test] X_snow_rvect_test = to_row_vector(X_snow_test);
    
    for (k in 1:K_reg) {          //  K_regxN * Nx1  T
        for (n in 1:N) {
            X_reg_snow[n,k] = X_snow_rvect[n] * X_reg[n,k];
    }  }
    
    // same, but for test. Should do this with a function...
    for (k in 1:K_reg) {
        for (n in 1:N_test) {
            X_reg_snow_test[n,k] = X_snow_rvect_test[n] * X_reg_test[n,k];
    }  }
}
parameters {
    // intercept was causing divergences and coef interpretation 
    // makes more sense without intercept: 
    // I don't expect change in base depth absent melting or snowfall
    vector<upper=0>[K_month] beta_mo;           // coefficients for melting
    vector<lower=0, upper=1>[K_reg] beta_reg_snow;       // coef for region x snow interaction
    real<lower=0> sigma;                        // must be +ve
    real<lower=0> sig_mos;                      // must be +ve
}
transformed parameters {
}
model {
    vector[N] mu;                       // y_hat
    sigma ~ cauchy(0, 10);              // half Cauchy
    sig_mos ~ cauchy(0, 20);
    for (n in 1:K_month) {
        beta_mo[n] ~ normal(0, sig_mos) T[,0]; // sample from normal, only -ve
    }
    // prior on snow columns is beta over [0,1]
    beta_reg_snow ~ beta(2.2, 3);         // reparameterize so this and snow are from beta dist
    mu = X_month*beta_mo + X_reg_snow*beta_reg_snow;
    y ~ normal(mu, sigma);
}
generated quantities{
    vector[N_test] y_test;
    for(n in 1:N_test) {
        y_test[n] = normal_rng(X_month_test[n]*beta_mo + 
                               X_reg_snow_test[n]*beta_reg_snow, sigma);
    }
}
"""

In [22]:
sm = pystan.StanModel(model_code=stan_model_str, model_name='stan_model')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL stan_model_9ef8f4c0b8e822c1507c3465e9c985ca NOW.


In [23]:
# avoid recompile if possible
with open(model_path, 'wb') as f:
    pickle.dump(sm, f)

In [24]:
fit = sm.sampling(data=stan_data, iter=2_000, chains=4, n_jobs=-1,
                  sample_file="../../data/processed/stan_samples",
                  control={'adapt_delta': 0.85, # p accepting posterior draw
                           'stepsize': 1,  # just starting stepsize
                          }, 
                  seed=42, verbose=True)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [25]:
# for overnight run
try:
    with open(result_path, 'wb') as f:
        pickle.dump(fit, f)
# reload saved objects if not reruning sampler
except NameError:
    with open(model_path, 'rb') as f:
        sm = pickle.load(f)
    with open(result_path, 'rb') as f:
        fit = pickle.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  after removing the cwd from sys.path.


## MCMC Diagnostics
We will want to check:
1. Model actually runs.
1. Good Mixing of Chains: (fix with stronger prior, reparameterization)
    1. $\hat{R}$ is 1.1 or under for all parameters.
    1. When n_eff / n_transitions < 0.001 the estimators that we use are often biased and can significantly overestimate the true effective sample size.
1. Check tree depth:
if threshold saturated, increase tree depth _control={max_treedepth: 15}_
1. 

_



In [68]:
fit_summary = fit.summary()
fit_summary.keys()

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'move_to_end',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [27]:
stan_utility.check_all_diagnostics(fit)

KeyboardInterrupt: ignored

In [ ]:
# fix brackets in col nmaes
fit_df = (fit.to_dataframe()
          .rename(columns=lambda x: x.replace("[", "_"))
          .rename(columns=lambda x: x.replace("]", "")))

fit_df.head()

### Check test set metrics

In [66]:
y_pred = pd.DataFrame(data=fit.extract(['y_test'], inc_warmup=False)['y_test'].T.mean(axis=1),
                      columns=['y_pred'])
test_results = pd.concat([y_test.reset_index(drop=True), y_pred], axis=1)

r2_score(y_true=test_results.delta_base, y_pred=test_results.y_pred, )

0.022973038159297965

## Visualization of results

In [ ]:
fit_az = az.from_pystan(posterior=fit,
                        dims={'beta_reg_snow': ['Coefficients_for_Snow_by_Region'],
                              'beta_mo': ['Melting_Coefficients_by_Month']},
                        coords={'Coefficients_for_Snow_by_Region': X_region.columns.values.tolist(),
                                'Melting_Coefficients_by_Month': [calendar.month_name[i+1] for i in range(12)]}
                        )
rc = {'plot.max_subplots': None}
az.rcParams.update(rc)
sns.set_style('whitegrid')
az.plot_trace(fit_az)

In [ ]:
# get region names without "region_"
region_names = [reg[7:] for reg in X_region.columns.values.tolist()]

region_betas_df = fit_df.filter(regex="reg", axis=1)
reg_cols = region_betas_df.columns
region_betas_df = (region_betas_df
                   .rename(columns={col: reg_name for col, reg_name 
                                    in zip(reg_cols, region_names)})
                   .melt(var_name="region"))
region_betas_df.head(2)

In [ ]:
plt.style.use('bmh')
fig = sns.kdeplot(x=region_betas_df.value, hue=region_betas_df.region, fill=True, cut=0, bw_adjust=.3)
plt.suptitle("Estimated Base Increase per Unit of Snowfall", fontsize=20)
plt.xlabel("Effect of Unit of Powder");

In [ ]:
plt.style.use('ggplot')
jazzcup = sns.blend_palette(vapeplot.palette("jazzcup"), n_colors=region_betas_df.region.unique().size)
f, ax = plt.subplots(figsize=(12, 8))
sort_order = region_betas_df.groupby(['region']).mean().sort_values(by='value', ascending=True).index

sns.violinplot(x='region', y='value', data=region_betas_df,
            order=sort_order, palette=jazzcup)

plt.title("Estimated Base Increase per Unit of Snowfall: Bayesian Model", fontsize=20)
plt.xlabel('Region')
plt.ylabel('Fraction of Full Unit of Powder');

In [ ]:
month_betas_df = fit_df.filter(like='beta_mo').melt(var_name="month")
month_betas_df = month_betas_df[month_betas_df.value > month_betas_df.value.quantile(.02)]
month_map = {f"beta_mo_{i}": calendar.month_abbr[i] for i in range(1, 13)}
#month_betas_df['month'] = pd.to_datetime(month_betas_df['month'].replace(month_map), format="%B").dt.month.astype('category')
month_betas_df['month'] = month_betas_df['month'].replace(month_map).astype('str')
month_betas_df.head()

In [ ]:
def plot_snow_betas(df, start_mo):
    fig, ax = plt.subplots()
    month_ordered = [mo for mo in calendar.month_abbr[1:] if mo in df.month.unique()]
    start_mo_ix = month_ordered.index(start_mo)
    month_ordered = month_ordered[start_mo_ix:] + month_ordered[:start_mo_ix]
    sns.boxplot(data=df, y='value', x='month', order=month_ordered,
                ax=ax, )
    ax.set_ylabel('Inches Melted per Day')
    ax.set_xlabel('Month')
    ax.set_title('Estimated Snow Melted per Day by Month');
plot_snow_betas(month_betas_df, "Jan")

These estimates are mostly expected, but there seems to be low melting amounts during summer...this can be explained when we realize that most of the values for May-November were interpolated. The averages aren't weighted by ski acreage, so the large number of small ski stations on the east coast & midwest with short seasons are disproportionately affecting these numbers. 

In [ ]:
interpo_ratios=(long_series_df
    .assign(month=lambda x: x.pseudo_ts.dt.month)
    .groupby('month')
    .apply(lambda x: x.basecol_interpolated.sum()/x.shape[0])
    .to_frame()
    .reset_index()
    .rename(columns={0:'ratio'})
)
fig, ax = plt.subplots()
sns.barplot(data=interpo_ratios, x='month', y='ratio', ax=ax)
plt.title('Fraction of Base observations that were interpolated', fontsize=15)
[plt.text((i-.17), value+.01, str(value)) for i, value in enumerate(interpo_ratios.ratio.round(2).to_numpy())]
months_xticks(ax);

In [ ]:
plot_snow_betas(month_betas_df[~month_betas_df.month.isin(['Jun', 'Jul', 'Aug', 'Sep', 'Oct'])], "Nov")